# Exercício de Treinamento com SQLite3 e Dados Falsos Gerados pelo `Faker`

Neste exercício, você irá praticar operações comuns em um banco de dados SQLite utilizando a biblioteca `sqlite3` e o `Faker` para gerar dados fictícios. O objetivo é criar, manipular e consultar um banco de dados de clientes.

## Contexto

Você está trabalhando para uma empresa que deseja gerenciar seus clientes. Cada cliente tem as seguintes informações:

- ID (gerado automaticamente)
- Nome completo
- Email
- Endereço
- Telefone
- Data de cadastro

## Tarefas

1. **Listar todos os clientes**

    Crie uma função que liste todos os clientes no banco de dados, exibindo as seguintes informações:

    - ID
    - Nome
    - Email
    - Telefone
    - Data de Cadastro

2. **Buscar cliente pelo email**: Implemente uma função que receba um email como parâmetro e retorne as informações completas do cliente.

3. **Buscar cliente por id**: Implemente uma função que receba um id como parâmetro e retorne as informações completas do cliente.

4. **Atualizar informações de um cliente:** Crie uma função que permita atualizar o endereço e telefone de um cliente identificado pelo seu ID.

5. **Deletar um cliente:** Implemente uma função que remova um cliente do banco de dados com base no seu ID.

6. **Listar clientes cadastrados em um período:** Implemente uma consulta que retorne todos os clientes cadastrados entre duas datas fornecidas.

7. **Implementar paginação:** Implemente uma função com paginação que retorne 5 registros por vez.

8. **Verificar emails duplicados:** Crie uma função que encontre emails duplicados no banco de dados.

9. **Exportar para CSV:** Implemente uma função que exporte todos os clientes para um arquivo CSV.



In [31]:
import sqlite3
from faker import Faker
from datetime import datetime, timedelta
from pathlib import Path
import random
import csv
import os

# configs
BD_URL = Path("data")/"clientes.db"
faker = Faker()

In [15]:
# PARTE I - FUNÇÕES PARA CRIAR E POPULAR BD

def conectar_bd():
    conn = sqlite3.connect(BD_URL)
    cursor = conn.cursor()
    return conn, cursor

def criar_tabela():
    conn, cursor = conectar_bd()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS clientes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            endereco TEXT,
            telefone TEXT,
            data_cadastro TEXT
        )
    ''')
    conn.commit()
    conn.close()


def popular_tabela(quantidade=10):
    conn, cursor = conectar_bd()
    for _ in range(quantidade):
        nome = faker.name()
        email = faker.email()
        endereco = faker.address()
        telefone = faker.phone_number()
        data_cadastro = (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute('''
            INSERT INTO clientes (nome, email, endereco, telefone, data_cadastro)
            VALUES (?, ?, ?, ?, ?)
        ''', (nome, email, endereco, telefone, data_cadastro))
    conn.commit()
    conn.close()



criar_tabela()
popular_tabela(20) 

In [16]:
# PARTE II - RESOLVENDO EXERCÍCIOS

## 1. Listagem de registros
def listar_clientes():
    conn, cursor = conectar_bd()
    cursor.execute('''
                SELECT id, nome, email, telefone, data_cadastro
                FROM clientes
                ''')
    clientes = cursor.fetchall()
    conn.close()
    for cliente in clientes:
        print(cliente)

listar_clientes()

(1, 'Joshua Long', 'aarroyo@example.net', '533-568-6814x0045', '2024-12-09 00:06:57')
(2, 'Gina Elliott', 'hughesandrea@example.org', '328.508.9877x061', '2024-12-05 00:06:57')
(3, 'Jacob Nguyen', 'paulgutierrez@example.org', '(881)810-8055x79831', '2024-11-13 00:06:57')
(4, 'Marc Mills', 'gonzalezjoseph@example.org', '7358891306', '2024-11-28 00:06:57')
(5, 'John Montgomery', 'cunninghamwilliam@example.com', '298-451-2149x3095', '2024-11-11 00:06:57')
(6, 'Michael Barnes', 'jacquelineknight@example.com', '(206)625-3061x83380', '2024-11-18 00:06:57')
(7, 'Melissa Macdonald', 'ogreen@example.org', '001-815-531-0338', '2024-12-10 00:06:57')
(8, 'Shelia Ray', 'ppowell@example.com', '657-303-3943x69105', '2024-11-25 00:06:57')
(9, 'Joe Myers', 'brandon16@example.com', '001-390-886-8921x2635', '2024-11-30 00:06:57')
(10, 'Grace Mendoza', 'derek06@example.org', '001-946-612-7067x6695', '2024-11-28 00:06:57')
(11, 'Paula Morrison', 'brandonwerner@example.com', '(868)486-3463x14719', '2024-12-

In [19]:
## 2. busca de um cliente por email
def buscar_cliente_email(email):
    conn, cursor = conectar_bd()
    cursor.execute('''
                SELECT *
                FROM clientes
                WHERE email = ?
                ''', (email,))
    cliente = cursor.fetchone()
    conn.close()
    return cliente

email = 'ogreen@example.org'
buscar_cliente_email(email)

(7,
 'Melissa Macdonald',
 'ogreen@example.org',
 '64675 Robert Squares\nTuckermouth, WI 36490',
 '001-815-531-0338',
 '2024-12-10 00:06:57')

In [20]:
## 3. Busca de um cliente por id
def buscar_cliente_id(id):
    conn, cursor = conectar_bd()
    cursor.execute('''
                SELECT *
                FROM clientes
                WHERE id = ?
                ''', (id,))
    cliente = cursor.fetchone()
    conn.close()
    return cliente

buscar_cliente_id(2)

(2,
 'Gina Elliott',
 'hughesandrea@example.org',
 '917 Matthew Ridges\nSouth Mistytown, NJ 82701',
 '328.508.9877x061',
 '2024-12-05 00:06:57')

In [21]:
## 4. atualização de um cliente
def atualizar_cliente(id, novo_endereco, novo_telefone):
    conn, cursor = conectar_bd()
    cursor.execute(
        '''
        UPDATE clientes
        SET endereco = ?, telefone = ?
        WHERE id = ?
        ''', 
        (novo_endereco, novo_telefone, id)
    )
    conn.commit()
    conn.close()

atualizar_cliente(2, 'Rua dos Bobos, 0', '000-000-000-0000')
buscar_cliente_id(2)

(2,
 'Gina Elliott',
 'hughesandrea@example.org',
 'Rua dos Bobos, 0',
 '000-000-000-0000',
 '2024-12-05 00:06:57')

In [22]:
## 5. exclusão de um cliente
def deletar_cliente(id):
    conn, cursor = conectar_bd()
    cursor.execute('DELETE FROM clientes WHERE id = ?', (id,))
    conn.commit()
    conn.close()

deletar_cliente(2)
if buscar_cliente_id(2) is None:
    print('Registro não encontrado')


Registro não encontrado


In [23]:
## 6. listagem de clientes por periodo
def listar_clientes_por_periodo(data_inicio, data_fim):
    conn, cursor = conectar_bd()
    cursor.execute('''
        SELECT id, nome, email, telefone, data_cadastro
        FROM clientes
        WHERE data_cadastro BETWEEN ? AND ?
    ''', (data_inicio, data_fim))
    clientes = cursor.fetchall()
    conn.close()
    for cliente in clientes:
        print(cliente)

data_inicio = "2024-12-01 00:00:00"
data_fim = "2024-12-10 23:59:59"
listar_clientes_por_periodo(data_inicio, data_fim)

(1, 'Joshua Long', 'aarroyo@example.net', '533-568-6814x0045', '2024-12-09 00:06:57')
(7, 'Melissa Macdonald', 'ogreen@example.org', '001-815-531-0338', '2024-12-10 00:06:57')
(11, 'Paula Morrison', 'brandonwerner@example.com', '(868)486-3463x14719', '2024-12-04 00:06:57')
(13, 'Susan Davis', 'joseph66@example.net', '001-762-834-2739x000', '2024-12-09 00:06:57')


In [24]:
## 7. busca com paginação
def lista_clientes_paginacao(pagina=1, tamanho=5):
    conn, cursor = conectar_bd()
    offset = (pagina-1)*tamanho
    cursor.execute('''
        SELECT id, nome, email, telefone, data_cadastro
        FROM clientes
        LIMIT ? OFFSET ?
    ''', (tamanho, offset))
    clientes = cursor.fetchall()
    conn.close()
    for cliente in clientes:
        print(cliente)

lista_clientes_paginacao(2, 5)

(7, 'Melissa Macdonald', 'ogreen@example.org', '001-815-531-0338', '2024-12-10 00:06:57')
(8, 'Shelia Ray', 'ppowell@example.com', '657-303-3943x69105', '2024-11-25 00:06:57')
(9, 'Joe Myers', 'brandon16@example.com', '001-390-886-8921x2635', '2024-11-30 00:06:57')
(10, 'Grace Mendoza', 'derek06@example.org', '001-946-612-7067x6695', '2024-11-28 00:06:57')
(11, 'Paula Morrison', 'brandonwerner@example.com', '(868)486-3463x14719', '2024-12-04 00:06:57')


In [27]:
# 8. busca de clientes com emails duplicados
def buscar_emails_duplicados():
    conn, cursor = conectar_bd()
    cursor.execute('''
        SELECT email, COUNT(*) as quantidade
        FROM clientes
        GROUP BY email
        HAVING COUNT(*) > 1
    ''')
    emails_duplicados = cursor.fetchall()
    conn.close()
    
    if emails_duplicados:
        print("Emails duplicados encontrados:")
        for email, quantidade in emails_duplicados:
            print(f"Email: {email}, Ocorrências: {quantidade}")
    else:
        print("Nenhum email duplicado encontrado.")
    
    return emails_duplicados

buscar_emails_duplicados()

Nenhum email duplicado encontrado.


[]

In [33]:
## 9. exportando para csv
def exportar_clientes_csv():
    conn, cursor = conectar_bd()
    cursor.execute('''
        SELECT id, nome, email, telefone, data_cadastro
        FROM clientes
    ''')
    clientes = cursor.fetchall()
    conn.close()
    
    directory = 'data'
    file_path = os.path.join(directory, 'clientes.csv')
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'nome', 'email', 'telefone', 'data_cadastro'])
        writer.writerows(clientes)
    
    print(f'Arquivo {file_path} criado com sucesso!')

exportar_clientes_csv()

Arquivo data/clientes.csv criado com sucesso!
